<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.00,"49,600.00",1,0
1,0,46.00,"38,000.00",1,1
2,0,29.00,"21,000.00",0,0
3,0,21.00,"41,700.00",2,0
4,1,28.00,"26,100.00",0,0


Проверим типы данных

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Преобразуем названия столбцов для удобства

In [5]:
df.columns = ['sex', 'age', 'salary', 'family_members', 'payments']

Заменим тип данных 'age' и 'salary' на int64

In [6]:
df['age'] = df['age'].astype('int')

In [7]:
df['salary'] = df['salary'].astype('int')

Проверим наличие пропусков

In [8]:
df.isna().sum()

sex               0
age               0
salary            0
family_members    0
payments          0
dtype: int64

Проверим наличие дубликатов

In [9]:
df.duplicated().sum()

153

Удалим дубликаты

In [10]:
df = df[~df.duplicated()]

Выделим признаки и целевой признак

In [11]:
target = df['payments']
features = df.drop(['payments'] , axis=1)

Напишим переиспользуемую функцию обучения и применения линейной регрессии

In [12]:
def lr_model(features, target):
    features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.25,
    random_state=17)
    lr = LinearRegression().fit(features_train, target_train)
    scores = cross_val_score(lr, features_train, target_train, scoring='r2', cv=5)
    print('Train scores:')
    print('scores = {} \nmean score = {:.5f} +/- {:.5f}'.format(scores, scores.mean(), scores.std()))
    print()
    print('Test scores:')
    scores = cross_val_score(lr, features_test, target_test, scoring='r2', cv=5)
    print('scores = {} \nmean score = {:.5f} +/- {:.5f}'.format(scores, scores.mean(), scores.std()))

Проверим оценки на трейне и тесте

In [13]:
lr_model(features, target)

Train scores:
scores = [0.44233097 0.4423793  0.41464155 0.41437853 0.43723501] 
mean score = 0.43019 +/- 0.01294

Test scores:
scores = [0.38492451 0.44838518 0.42374175 0.41821723 0.40131333] 
mean score = 0.41532 +/- 0.02141


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Требования к матрице на которую домножаем признаки: соответствующая размерность для домножения (те должна быть согласована) и наличие обратной матрицы.

Размерность исходной матрицы

In [14]:
features.shape

(4847, 4)

Сгенерируем матрицу для домножения

In [15]:
coder = np.random.rand(4,4)
coder

array([[0.89856913, 0.0777602 , 0.66925436, 0.22883809],
       [0.92418796, 0.52327007, 0.99199601, 0.11175195],
       [0.1311682 , 0.09634312, 0.37395671, 0.35521436],
       [0.42059163, 0.58314951, 0.32840864, 0.45462248]])

Проведем операцию матричного умножения

In [16]:
features_coded = features.dot(coder)

In [17]:
features_coded

,0,1,2,3
0,"6,545.15","4,800.73","18,589.92","17,623.90"
1,"5,027.32","3,685.69","14,256.32","13,503.74"
2,"2,781.33","2,038.38","7,881.86","7,462.74"
3,"5,489.96","4,029.66","15,615.48","14,815.69"
4,"3,450.27","2,529.28","9,788.72","9,274.45"
...,...,...,...,...
4995,"4,709.42","3,455.27","13,378.69","12,685.19"
4996,"6,905.06","5,066.75","19,629.39","18,617.49"
4997,"4,465.93","3,277.66","12,697.63","12,044.91"
4998,"4,311.69","3,163.76","12,251.86","11,619.56"


Проверим оценки модели на получившихся фичах

In [18]:
lr_model(features_coded, target)

Train scores:
scores = [0.44233097 0.4423793  0.41464155 0.41437853 0.43723501] 
mean score = 0.43019 +/- 0.01294

Test scores:
scores = [0.38492451 0.44838518 0.42374175 0.41821723 0.40131333] 
mean score = 0.41532 +/- 0.02141


Оценки модели не изменились

**Обоснование:**

Если матрицу признаков домножить на любую обратимую матрицу, то результат работы модели не изменится. Главное умножать все признаки поступающие в модель на обучение и предсказания на одинаковую шифрующую матрицу.

Домножение целевого таргета зависит от необходимости шифрования ответов

Цель обучения линейной регрессии - уменьшить среднеквадратичную ошибку между признаками умноженными на найденные веса и целевым признаком: 
$
w = \arg\min_w MSE(Xw, y)
$

Формула обучения: 
$
w = (X^T X)^{-1} X^T y
$

Веса мы находим по трейн признакам и трейн таргетам
Так что если линейно изменить признаки домножением на матрицу, то алгоритм просто изменит веса в соответсвии с получившимися признаками

Формула предсказания: 
$
a = Xw
$

Предсказания по шифрованным признакам будут правильными


## Алгоритм преобразования

**Алгоритм**

Так как при домножении признаков на обратимую матрицу качество линейной регрессии не меняется, то анонимировать данные будем следующим алгоритмом:
Используем размерность исходной матрицы и правила перемножения матриц для генерации матрицы-преобразователя.
1. Так как нужно защитить данные, то матрица обязана быть случайной np.random.rand().
2. Сохраним ключ для повторного и обратного преобразования np.random.RandomState(key), в виде RandomState.

**Обоснование**

Домножение признаков на обратимую матрицу качество линейной регрессии не меняет

## Проверка алгоритма

Напишем функцию - шифратор

In [19]:
def anonimaizer(features):
    key = 17
    np.random.RandomState(key)
    i = features.shape[1]
    coder = np.random.rand(i,i)
    features_coded = features.dot(coder)
    return features_coded

In [20]:
features_coded = anonimaizer(features)

Проверим оценки метрики

In [21]:
lr_model(features_coded, target)

Train scores:
scores = [0.44233097 0.4423793  0.41464155 0.41437853 0.43723501] 
mean score = 0.43019 +/- 0.01294

Test scores:
scores = [0.38492451 0.44838518 0.42374175 0.41821723 0.40131333] 
mean score = 0.41532 +/- 0.02141
